![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 

## Company's description 📇

<a href="https://www.kayak.com" target="_blank">Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href="https://www.bookingholdings.com/" target="_blank">Booking Holdings</a> which now holds: 

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

In [1]:
import os
import json
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, UTC
import boto3
import psycopg2
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, text

In [2]:
top_35_France = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

# Get weather data with an API 

## API localisation with nominatim.org

In [7]:
Info_Cities = pd.DataFrame(columns = ['city_name', 'city_id', 'latitude', 'longitude'])

header = {'User-Agent': 'Formation'}
for i in range(len(top_35_France)):
    payload = {'q': top_35_France[i], 'format': 'json'}
    r = requests.get(url="https://nominatim.openstreetmap.org/search?", params = payload, headers=header)
    Info_Cities.loc[i] = [top_35_France[i], i+1, r.json()[0]['lat'], r.json()[0]['lon']]
Info_Cities.head()

,city_name,city_id,latitude,longitude
0,Mont Saint Michel,1,48.6359541,-1.511459954959514
1,St Malo,2,49.314695,-96.9538228
2,Bayeux,3,49.2764624,-0.7024738
3,Le Havre,4,49.4938975,0.1079732
4,Rouen,5,49.4404591,1.0939658


In [8]:
print(Info_Cities.shape)
# Sauvegarde dans un fichier json
Info_Cities.to_json('src/35Cities_Info.json')

(35, 4)


## Weather predictions (7 days) from openweathermap.org

In [9]:
Weather_Cities = pd.DataFrame()

from keys import WEATHER_KEY

for id_ville in range(len(top_35_France)):
    city_name = top_35_France[id_ville]
    city_id = id_ville+1
    payload = {'lat': Info_Cities.loc[id_ville,"latitude"], 
           'lon': Info_Cities.loc[id_ville,"longitude"], 
           'appid': WEATHER_KEY,
           'units': "metric",
            'exclude': 'current,minutely,hourly,alerts' 
           }
    r = requests.get(url="https://api.openweathermap.org/data/3.0/onecall", params = payload)
    dict_weather = r.json()
    for i in range(len(dict_weather['daily'])):
        date = datetime.fromtimestamp(dict_weather['daily'][i]['dt'], UTC).strftime('%Y-%m-%d')
        temp = dict_weather['daily'][i]['temp']['day'] 
        humidity = dict_weather['daily'][i]['humidity'] 
        wind = dict_weather['daily'][i]['wind_deg'] 
        type_weather = dict_weather['daily'][i]['weather'][0]['main'] 
        Weather_Cities = pd.concat([Weather_Cities, pd.DataFrame([city_name, city_id, date, temp, 
                                                                  humidity, wind, type_weather]).T], 
                                                                  ignore_index=True)
Weather_Cities.columns = ['city_name', 'city_id', 'date', 'temp', 'humidity', 'wind', 'type_weather']
Weather_Cities

,city_name,city_id,date,temp,humidity,wind,type_weather
0,Mont Saint Michel,1,2025-03-28,9.74,89,316,Rain
1,Mont Saint Michel,1,2025-03-29,10.61,66,334,Clouds
2,Mont Saint Michel,1,2025-03-30,13.45,78,315,Rain
3,Mont Saint Michel,1,2025-03-31,15.61,58,76,Clear
4,Mont Saint Michel,1,2025-04-01,14.23,52,94,Clear
...,...,...,...,...,...,...,...
275,La Rochelle,35,2025-03-31,14.26,62,72,Clear
276,La Rochelle,35,2025-04-01,13.89,58,75,Clear
277,La Rochelle,35,2025-04-02,14.37,55,83,Rain
278,La Rochelle,35,2025-04-03,15.73,70,135,Rain


In [10]:
# Sauvegarde dans un fichier json

Weather_Cities.to_json('src/35Cities_Weather.json')

# Scrape Booking.com 

In [118]:
# Lancer le scraping pour chacune des 35 villes

for ville in top_35_France:
    os.system(f"python Scraping/Booking.py \"{ville}\"")

In [69]:
# Créer un fichier csv unique concaténant les données weather et hotels 

df_weather = pd.read_json("src/35Cities_Weather.json").set_index('city_id')
liste_hotels = []
for i in range(len(top_35_France)):
    df_hotel = pd.read_json("src/result_booking_"+str(top_35_France[i])+".json")
    df_hotel['city_id'] = [i+1 for j in df_hotel.index]
    df_hotel['hotel_id'] = [j+1 for j in df_hotel.index]
    liste_hotels.append(df_hotel)

df_hotels = pd.merge(liste_hotels[0], liste_hotels[1], how='outer')
for i in range(len(liste_hotels)-2):
    df_hotels = pd.merge(df_hotels, liste_hotels[i+2], how='outer')

df_concat = pd.merge(df_weather, df_hotels, on='city_id', how='left')
df_concat.to_json('src/35Cities_Weather_Hotels.json')

In [70]:
df_concat.shape
# 8 jours * 766 hôtels : OK

(6128, 14)

# Create your data lake using S3 

In [2]:
from keys import AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY

session = boto3.Session(aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
s3 = session.resource('s3')
CreateBucketConfiguration={
    'LocationConstraint': 'eu-west-3',
}
bucket = s3.create_bucket(Bucket="eugenie-s3-jedha-project-kayak")
bucket.upload_file('./src/35Cities_Weather_Hotels.json', '35Cities')

# Figures

## Map of top-5 destinations according to the weather

In [ ]:
df = pd.read_json("https://eugenie-s3-jedha-project-kayak.s3.us-east-1.amazonaws.com/35Cities")
df.head()

,city_id,city_name,date,temp,humidity,wind,type_weather,name,latitude,longitude,score,url,description,hotel_id
0,1,Mont Saint Michel,2025-03-28,9.74,89,316,Rain,Apparthôtel Mont Saint Michel - Résidence Fleu...,48.596482,-1.503147,8.2,https://www.booking.com/hotel/fr/residence-fle...,L’Apparthôtel Mont Saint Michel - Résidence Fl...,4
1,1,Mont Saint Michel,2025-03-28,9.74,89,316,Rain,Auberge Saint Pierre,48.635688,-1.509883,8.1,https://www.booking.com/hotel/fr/auberge-saint...,The Auberge Saint Pierre is made up of small 1...,11
2,1,Mont Saint Michel,2025-03-28,9.74,89,316,Rain,Chez Adèle,48.636166,-1.510476,8.3,https://www.booking.com/hotel/fr/chez-adele-le...,"Doté d&#x27;installations de sports nautiques,...",12
3,1,Mont Saint Michel,2025-03-28,9.74,89,316,Rain,Gites Bellevue,48.607880,-1.517224,9.2,https://www.booking.com/hotel/fr/gites-bellevu...,Situé à seulement 2 km du célèbre Mont-Saint-M...,14
4,1,Mont Saint Michel,2025-03-28,9.74,89,316,Rain,Gites les 2 Clos au pied du Mont,48.613966,-1.485792,9.1,https://www.booking.com/hotel/fr/les-deux-clos...,L&#x27;établissement Gites les 2 Clos au pied ...,6


In [ ]:
df_weather = df.loc[df['hotel_id']==1, :]
df_weather_7day = df_weather.loc[df_weather['date']=='2025-04-04', :]
display(df_weather_7day.head())
df_weather_7day.shape

,city_id,city_name,date,temp,humidity,wind,type_weather,name,latitude,longitude,score,url,description,hotel_id
152,1,Mont Saint Michel,2025-04-04,17.87,60,144,Clouds,Les Terrasses Poulard,48.635349,-1.510379,7.4,https://www.booking.com/hotel/fr/les-terrasses...,Occupant 2 bâtiments différents au cœur du Mon...,1
351,2,St Malo,2025-04-04,1.27,99,177,Snow,Malouinière Guillaume Onffroy,48.632509,-1.986458,9.5,https://www.booking.com/hotel/fr/malouiniere-g...,"Situé à Saint-Malo, à seulement 2,4 km de la p...",1
509,3,Bayeux,2025-04-04,18.27,62,125,Clouds,B&amp;B Nathalie,49.276465,-0.706367,9.5,https://www.booking.com/hotel/fr/imagine-bayeu...,Doté d&#x27;une terrasse et d&#x27;un salon co...,1
679,4,Le Havre,2025-04-04,16.58,64,127,Clouds,Hilton Garden Inn Le Havre Centre,49.490087,0.097747,8.7,https://www.booking.com/hotel/fr/hilton-garden...,Le Hilton Garden Inn Le Havre Centre est situé...,1
882,5,Rouen,2025-04-04,20.12,53,122,Clear,Studio N1 hypercentre Rue Saint Nicolas,49.440823,1.097391,8.5,https://www.booking.com/hotel/fr/studio-hyperc...,Le Studio N1 hypercentre Rue Saint Nicolas pro...,1


(35, 14)

In [ ]:
fig = go.Figure()

categories = df_weather_7day.columns[3:6].to_list()
buttons = []

for i in range(0, 3):
    fig.add_trace(
        go.Scattergeo(
            lat = df_weather_7day['latitude'],
            lon = df_weather_7day['longitude'],
            text = df_weather_7day['city_name'],
            mode = 'markers',
            marker = dict(
                opacity = 0.8,
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'
                ),
                colorscale = 'Jet',
                cmin = 0,
                color = df_weather_7day[categories[i]],
                cmax = df_weather_7day[categories[i]].max(),
                colorbar=dict(
                    title=dict(
                        text=categories[i]
                    )
                )),
            visible=(i==0)
            )
    )
    buttons.append(go.layout.updatemenu.Button(
                    label = categories[i],
                    method = "update",
                    args = [{"visible" : [i==0, i==1, i==2]}]))


fig.update_layout(
    title = go.layout.Title(text = "Previsions for 35 cities in 7 days", x = 0.5),
    showlegend = False,
    width=700, 
    updatemenus = [go.layout.Updatemenu(
        active = 0,
        buttons = buttons
    )]
)

fig.show()

In [ ]:
fig = px.scatter_mapbox(df_weather, lat="latitude", lon="longitude", color="temp", size="humidity",
                        center=dict(lat=df_weather["latitude"].mean(), lon=df_weather["longitude"].mean()), 
                        zoom=4, mapbox_style="open-street-map", width=600, height=600, 
                        color_continuous_scale = "Jet", hover_name='city_name',
                        animation_frame = "date", range_color=[df_weather['temp'].min(), df_weather['temp'].max()],
                        title='Temperature and humidity (size) for 35 cities in the next 7 days')
fig.show()

## Map of top-20 hotels in the 5 best cities

In [ ]:
best5_cities = df_weather_7day.loc[df_weather_7day['temp']>19,:]\
    .loc[df_weather_7day['humidity']<50,:]\
    .loc[df_weather_7day['wind']<150,"city_name"].to_list()
print(best5_cities)

df_hotels = df.loc[df['city_name'].isin(best5_cities), :].loc[df['date'] == '2025-04-04', :]
df_hotels['short_description1'] = [x[:100] for x in df_hotels['description']]
df_hotels['short_description2'] = [x[100:200] for x in df_hotels['description']]
df_hotels['short_description3'] = [x[200:300] for x in df_hotels['description']]
print(df_hotels.shape)
df_hotels.head()


['Paris', 'Lille', 'Besancon', 'Grenoble', 'Toulouse']
(108, 17)


,city_id,city_name,date,temp,humidity,wind,type_weather,name,latitude,longitude,score,url,description,hotel_id,short_description1,short_description2,short_description3
1028,6,Paris,2025-04-04,22.06,38,114,Clear,Appart Merryland Paris Châtelet,48.860978,2.348225,9.1,https://www.booking.com/hotel/fr/appartement-p...,"Set in the centre of Paris, Appart Merryland P...",14,"Set in the centre of Paris, Appart Merryland P...",historic building. This recently renovated ap...,d 800 metres from Sainte-Chapelle. Orsay Museu...
1029,6,Paris,2025-04-04,22.06,38,114,Clear,Appartement Paris 16 proche à 20 minutes de la...,48.838346,2.260577,9.3,https://www.booking.com/hotel/fr/elegance-pari...,"Offrant une vue sur la cour intérieure, l&#x27...",5,"Offrant une vue sur la cour intérieure, l&#x27...","iffel est situé à Paris, à 4,1 km de la tour E...","de Versailles. Il propose un ascenseur, un ser..."
1030,6,Paris,2025-04-04,22.06,38,114,Clear,Appartement parisien baigné de soleil,48.883823,2.292958,8.3,https://www.booking.com/hotel/fr/appartement-p...,L&#x27;Appartement parisien baigné de soleil e...,8,L&#x27;Appartement parisien baigné de soleil e...,"z à 1,3 km du palais des congrès de Paris, à 3...","fel. Vous séjournerez à 3,5 km de l&#x27;opéra..."
1031,6,Paris,2025-04-04,22.06,38,114,Clear,BESTInPARIS,48.860363,2.347777,9.3,https://www.booking.com/hotel/fr/bestinparis.f...,"Situé à Paris, à moins de 1 km du musée du Lou...",20,"Situé à Paris, à moins de 1 km du musée du Lou...",", le BESTInPARIS propose la climatisation. Il ...","4,1 km du musée d&#x27;Orsay et à 4,4 km de l&..."
1032,6,Paris,2025-04-04,22.06,38,114,Clear,"Charming Studio Ideally Located in Paris, 9th",48.882512,2.333105,8.7,https://www.booking.com/hotel/fr/charming-stud...,"Offrant une vue sur la ville, le Charming Stud...",12,"Offrant une vue sur la ville, le Charming Stud...","tué à Paris, à 1,2 km de la gare Saint-Lazare ...",d&#x27;un service de sécurité d&#x27;une jour...


In [ ]:
for ville in best5_cities:
    fig = px.scatter_mapbox(df_hotels.loc[df_hotels['city_name']==ville], lat="latitude", lon="longitude", 
                color='score', width=600, height=600, zoom=11,  
                mapbox_style='carto-positron', 
                custom_data=['short_description1', 'short_description2', 'short_description3'],
                title='Scores and description of the hotels found in '+str(ville)
                )
    fig.update_traces(hovertemplate='<b>Description: </b><br> %{customdata[0]} <br>'+
                  ' %{customdata[1]} <br> %{customdata[2]}') 
    fig.show()

# ETL

## Extract

In [2]:
from keys import PWD_DB

engine = create_engine(f"postgresql+psycopg2://postgres:"+PWD_DB+
                       "@database-kayak.cviskoi684b5.us-east-1.rds.amazonaws.com:5432/postgres", 
                       echo=True)

Base = declarative_base()
Base.metadata.create_all(engine)

/tmp/ipykernel_319806/829915135.py:9: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


2025-04-05 00:25:10,346 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-04-05 00:25:10,348 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-05 00:25:10,533 INFO sqlalchemy.engine.Engine select current_schema()
2025-04-05 00:25:10,534 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-05 00:25:10,720 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-04-05 00:25:10,720 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-05 00:25:10,906 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-05 00:25:10,907 INFO sqlalchemy.engine.Engine COMMIT


## Transform

In [3]:
dataset = pd.read_json("https://eugenie-s3-jedha-project-kayak.s3.us-east-1.amazonaws.com/35Cities")
dataset.head()

,city_id,city_name,date,temp,humidity,wind,type_weather,name,latitude,longitude,score,url,description,hotel_id
0,1,Mont Saint Michel,2025-03-28,9.74,89,316,Rain,Apparthôtel Mont Saint Michel - Résidence Fleu...,48.596482,-1.503147,8.2,https://www.booking.com/hotel/fr/residence-fle...,L’Apparthôtel Mont Saint Michel - Résidence Fl...,4
1,1,Mont Saint Michel,2025-03-28,9.74,89,316,Rain,Auberge Saint Pierre,48.635688,-1.509883,8.1,https://www.booking.com/hotel/fr/auberge-saint...,The Auberge Saint Pierre is made up of small 1...,11
2,1,Mont Saint Michel,2025-03-28,9.74,89,316,Rain,Chez Adèle,48.636166,-1.510476,8.3,https://www.booking.com/hotel/fr/chez-adele-le...,"Doté d&#x27;installations de sports nautiques,...",12
3,1,Mont Saint Michel,2025-03-28,9.74,89,316,Rain,Gites Bellevue,48.607880,-1.517224,9.2,https://www.booking.com/hotel/fr/gites-bellevu...,Situé à seulement 2 km du célèbre Mont-Saint-M...,14
4,1,Mont Saint Michel,2025-03-28,9.74,89,316,Rain,Gites les 2 Clos au pied du Mont,48.613966,-1.485792,9.1,https://www.booking.com/hotel/fr/les-deux-clos...,L&#x27;établissement Gites les 2 Clos au pied ...,6


In [4]:
for colonne in ['name', 'description']:
    dataset[colonne] = [x.replace("&#x27;","'") for x in dataset[colonne]]
    dataset[colonne] = [x.replace("&amp;","et") for x in dataset[colonne]]
    dataset[colonne] = [x.replace("\u00e9;","é") for x in dataset[colonne]]
    dataset[colonne] = [x.replace("\u00e8;","è") for x in dataset[colonne]]
    dataset[colonne] = [x.replace("\u00e0;","à") for x in dataset[colonne]]
    dataset[colonne] = [x.replace("\u00e2;","â") for x in dataset[colonne]]
    dataset[colonne] = [x.replace("\u00f4;","ô") for x in dataset[colonne]]

In [5]:
df_weather = dataset.loc[dataset['hotel_id']==1, :]
df_hotels = dataset.loc[dataset['date'] == '2025-04-04', :]

## Load

In [6]:
df_weather.to_sql(
    "citiesweather",
    engine,
    if_exists="replace"
)

2025-04-05 00:25:24,724 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-05 00:25:24,750 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-04-05 00:25:24,753 INFO sqlalchemy.engine.Engine [generated in 0.00312s] {'table_name': 'citiesweather', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-04-05 00:25:24,941 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalo

280

In [7]:
df_hotels.to_sql(
    "citieshotels",
    engine,
    if_exists="replace"
)

2025-04-05 00:25:30,893 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-05 00:25:30,904 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-04-05 00:25:30,906 INFO sqlalchemy.engine.Engine [cached since 6.156s ago] {'table_name': 'citieshotels', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-04-05 00:25:31,101 INFO sqlalchemy.engine.Engine 
CREATE TABLE citieshotels (
	index BIGINT, 
	city_id BIGINT, 
	city_name TEXT, 
	date TIMESTAMP WITHOUT TIME ZONE, 
	temp FLOAT(53), 
	humidity BIGINT, 
	wind BIGINT, 
	t

766

In [8]:
# requête sur les bases de données créées, pour vérifier que tout est OK

conn = engine.connect()
statement = text("SELECT DISTINCT date FROM citiesweather")
result = conn.execute(statement)
result.fetchall()

2025-04-05 00:25:37,518 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-05 00:25:37,523 INFO sqlalchemy.engine.Engine SELECT DISTINCT date FROM citiesweather
2025-04-05 00:25:37,527 INFO sqlalchemy.engine.Engine [generated in 0.00879s] {}


[(datetime.datetime(2025, 3, 31, 0, 0),),
 (datetime.datetime(2025, 4, 4, 0, 0),),
 (datetime.datetime(2025, 4, 3, 0, 0),),
 (datetime.datetime(2025, 4, 2, 0, 0),),
 (datetime.datetime(2025, 3, 28, 0, 0),),
 (datetime.datetime(2025, 4, 1, 0, 0),),
 (datetime.datetime(2025, 3, 29, 0, 0),),
 (datetime.datetime(2025, 3, 30, 0, 0),)]

In [9]:
statement = text("SELECT DISTINCT score FROM citieshotels")
result = conn.execute(statement)
result.fetchall()

2025-04-05 00:25:40,366 INFO sqlalchemy.engine.Engine SELECT DISTINCT score FROM citieshotels
2025-04-05 00:25:40,368 INFO sqlalchemy.engine.Engine [generated in 0.00170s] {}


[(7.7,),
 (8.7,),
 (9.5,),
 (7.8,),
 (8.9,),
 (9.8,),
 (8.3,),
 (8.2,),
 (9.6,),
 (8.6,),
 (9.9,),
 (9.3,),
 (7.2,),
 (6.9,),
 (6.6,),
 (9.2,),
 (7.9,),
 (9.7,),
 (7.6,),
 (7.1,),
 (9.1,),
 (4.5,),
 (8.1,),
 (8.8,),
 (7.4,),
 (7.3,),
 (7.5,),
 (9.4,),
 (8.5,),
 (8.4,)]